In [5]:
import os
import shutil
import re
import itertools as it
os.chdir("/Users/BeckyMarcusMacbook/Thesis/manual_work/")
text_dir = "data/groups/E3/texts"
dest_dir = "E3_teset"
def reset():
    rm_ds(text_dir)
    if os.path.exists(dest_dir):
        shutil.rmtree(dest_dir)
    shutil.copytree(text_dir,dest_dir)
def rm_ds(dir):
    for file in os.listdir(dir):
        if file==".DS_Store":
            os.remove(os.path.join(dir,file))

In [7]:
def make_dirs_by_page(read_dir:str,write_dir:str):
    for file in os.listdir(read_dir):
        source_path = os.path.join(read_dir,file)
        page_num = file[-6:-4]
        dest_dir = os.path.join(write_dir,page_num)
        os.makedirs(dest_dir,exist_ok=True)
        shutil.copyfile(source_path,os.path.join(dest_dir,file))
temp_num_dir = 'data/temp/E3_pages'
shutil.rmtree(temp_num_dir)
make_dirs_by_page(text_dir,temp_num_dir)      
        
        

In [5]:
text_dir

'data/groups/E2/texts'

- always remove page 0
### Title
- Until 1912: can look for first time with two lines with 3 or more lowercase
- 1912 on: take up to and including line with entire lowercase word

### Footnotes: 
- II on 1912 p.1, 2,3,4 on next page, 's ' on next page instead of 5, then IIra
- often time single number
- 1908-1 has a single I on a line
- 1910 p.13 has a '1 ' before footnote
- 1915 uses I, ', '4 '

- always first line on non-title pages


In [6]:
def handle_first_page(file:str, text:str)->str:
# Until 1912: can look for first time with two lines with 3 or more lowercase
#1912 on: take up to and including line with entire lowercase word
    disc,year,num,pagetxt = file.split("-")
    lines = text.split("\n")
    line_num = 0
    if int(year)<1912:
        num_lowercase_lines =0
        while num_lowercase_lines<2:
            line = lines[line_num]
            if re.search(r"[a-z]{3,}",line) is not None:
                num_lowercase_lines+=1
            else:
                num_lowercase_lines = 0
            line_num+=1
        start_line = line_num-1  
    else:
        while re.search(r"\b[a-z]+\b", lines[line_num]) is None:
            line_num+=1
        start_line = line_num+1
    return "\n".join(lines[start_line:])
def clean_headers_footers(dest_dir):
    file:str
    for file in sorted(os.listdir(dest_dir)):
        if file[0]=='.':
            continue
        disc,year,num,pagetxt = file.split("-")
        page=int(pagetxt[:-4])
        path = os.path.join(dest_dir,file)
        
        text = open(path,'r').read()
        
        lines = text.split("\n")
        lines_stripped = [l.strip() for l in lines]
        text = "\n".join(lines_stripped)
        
        jstor_split = text.split("This content downloaded from")
        text =jstor_split[0]
        if page>1:
            lines = text.split("\n")
            no_header = lines[1:]
            text = "\n".join(no_header)
        else:
            text = handle_first_page(file,text)
        with open(path,'w') as f:
            f.write(text.strip())
def is_first_page(file_name:str)->bool:
    pagetxt = file_name.split("-")[-1]
    return int(pagetxt[:-4]) ==0
    
def remove_pages(dir_path: str,decision_func):
    try:
        files = sorted(os.listdir(dir_path))
        for file in files:
            if decision_func(file):
                os.remove(os.path.join(dir_path, file))
    except Exception as e:
        raise
reset()
remove_pages(dest_dir,is_first_page)
clean_headers_footers(dest_dir)

In [25]:
footnotes_1912= ['01IImperial','032System','04s F','06IIra',"07'J.",'08"The','099See',"121' F"]
# 1913 starts with ' at the beginning

d = {}
for f in footnotes_1912:
    num = f[:2]
    word = f[2:]
    d[num] = word
d

{'01': 'IImperial',
 '03': '2System',
 '04': 's F',
 '06': 'IIra',
 '07': "'J.",
 '08': '"The',
 '09': '9See',
 '12': "1' F"}

In [4]:
E3_1912_FOOTNOTES = {'01': 'IImperial',
 '03': '2System',
 '04': 's F',
 '06': 'IIra',
 '07': "'J.",
 '08': '"The',
 '09': '9See',
 '12': "1' F"}

In [22]:
### Deal with line breaks across pages
files = os.listdir(dest_dir)
files.sort()
for first,second in it.pairwise(files):
    if first.split("-")[:3] != second.split("-")[:3]:
        continue
    path1 = os.path.join(dest_dir, first)
    path2 = os.path.join(dest_dir, second)

    with open(path1, 'r') as f1, open(path2, 'r') as f2:
        text1 = f1.read()
        text2 = f2.read()

    if text1.endswith("-"):
        first_word = re.match(r"^\S+", text2).group()
        new_text2 = re.sub(r"^\S+\s", "", text2)
        new_text1 = text1[:-1] + first_word

        with open(path1, 'w') as f1, open(path2, 'w') as f2:
            f1.write(new_text1.strip())
            f2.write(new_text2.strip())


In [23]:
### remove dash errors
def fix_dash_errors(text):
    new_text = re.sub(r"([a-zA-Z]+)\-+\n([a-zA-Z]+)([^\w\n\s])?", # Captures 3 groups: first half of word, second half of word, optional punctuation
                      r"\1\2\3\n", #removes dash and moves line break
                      text)
    new_text_lines_stripped=[line.strip() for line in new_text.split('\n')] #remove any extra leading or trailing whitespace
    return "\n".join(new_text_lines_stripped) #join lines back together
files = os.listdir(dest_dir)
files.sort()
for file in files:
    if file[0]=='.':
        continue
    path = os.path.join(dest_dir,file)
    text = open(path,'r').read()
    new_text = fix_dash_errors(text)
    open(path,'w').write(new_text)

### Handling footnotes

In [ ]:
### Footnotes: 
# - II on 1912 p.1, 2,3,4 on next page, 's ' on next page instead of 5, then IIra
# - often time single number
# - 1908-1 has a single I on a line
# - 1910 p.13 has a '1 ' before footnote
# - 1915 uses I, ', '4 '

In [27]:
foot_lines = {}
step_temp = "manual_work/E3/.footnote_orig"
if os.path.exists(step_temp):
    shutil.rmtree(step_temp)
os.makedirs(step_temp)
files = os.listdir(dest_dir)
files.sort()
for file in files:
    fn_on_page = False
    path = os.path.join(dest_dir,file)
    save_path = os.path.join(step_temp,file)
    text = open(path,'r').read()
    lines = text.split("\n")
    if "1912-0" in file:
        page_num = file[-6:-4]
        fn_line_start = E3_1912_FOOTNOTES.get(page_num)
        if fn_line_start is not None:
            for i,line in enumerate(lines):
                if re.match(fn_line_start,line):
                    if not fn_on_page:
                        fn_on_page = True
                        foot_lines[file]=[]
                    lines[i] = f"{i}-->{line}"
                    foot_lines[file].append(i)
                    break
    elif file =='Economics-1915-0-05.txt':
        #special case with weird start
        for i,line in enumerate(lines):
            if re.search(r"I Cf\.",line) is not None:
                fn_on_page = True
                lines[i] = f"{i}-->{line}"
                foot_lines[file] = [i]    
    else:
        for i,line in enumerate(lines):
            if re.search(r"^\'|^\d",line) is not None or len(line)<3:
                
                if not fn_on_page:
                    fn_on_page = True
                    foot_lines[file]=[]
                lines[i] = f"{i}-->{line}"
                foot_lines[file].append(i)
    if fn_on_page:
        new_text = "\n".join(lines)
        with open(save_path,'w') as f:
            f.write(new_text)

In [28]:
foot_lines

{'Economics-1907-0-03.txt': [12, 16, 20],
 'Economics-1907-0-10.txt': [29],
 'Economics-1907-0-14.txt': [26],
 'Economics-1907-0-17.txt': [34],
 'Economics-1908-0-01.txt': [28],
 'Economics-1908-1-01.txt': [23],
 'Economics-1908-1-02.txt': [12],
 'Economics-1908-1-03.txt': [5],
 'Economics-1908-1-04.txt': [2],
 'Economics-1908-1-05.txt': [2],
 'Economics-1908-1-06.txt': [7],
 'Economics-1908-1-12.txt': [8, 10, 13, 28],
 'Economics-1908-1-14.txt': [16],
 'Economics-1908-1-15.txt': [22],
 'Economics-1908-1-16.txt': [7],
 'Economics-1910-0-01.txt': [28],
 'Economics-1910-0-13.txt': [34],
 'Economics-1911-0-21.txt': [5],
 'Economics-1911-0-24.txt': [31],
 'Economics-1912-0-01.txt': [33],
 'Economics-1912-0-03.txt': [37],
 'Economics-1912-0-04.txt': [39],
 'Economics-1912-0-06.txt': [38],
 'Economics-1912-0-07.txt': [38],
 'Economics-1912-0-08.txt': [36],
 'Economics-1912-0-09.txt': [39],
 'Economics-1912-0-12.txt': [38],
 'Economics-1913-0-05.txt': [3, 28],
 'Economics-1913-0-09.txt': [4, 

In [7]:
edited_foot_lines = {
 'Economics-1907-0-14.txt': [26],
 'Economics-1907-0-17.txt': [34],
 'Economics-1908-0-01.txt': [28],
 'Economics-1908-1-01.txt': [23],
 'Economics-1910-0-01.txt': [28],
 'Economics-1910-0-13.txt': [34],
 'Economics-1911-0-24.txt': [31],
 'Economics-1912-0-01.txt': [33],
 'Economics-1912-0-03.txt': [37],
 'Economics-1912-0-04.txt': [39],
 'Economics-1912-0-06.txt': [38],
 'Economics-1912-0-07.txt': [38],
 'Economics-1912-0-08.txt': [36],
 'Economics-1912-0-09.txt': [39],
 'Economics-1912-0-12.txt': [38],
 'Economics-1913-0-09.txt': [34],
 'Economics-1913-0-12.txt': [38],
 'Economics-1915-0-05.txt': [38],
 'Economics-1915-0-14.txt': [40],
 'Economics-1915-0-16.txt': [37, 38, 39],
 'Economics-1915-0-21.txt': [14],
 'Economics-1916-0-01.txt': [31],
 'Economics-1916-0-02.txt': [40],
 'Economics-1916-0-04.txt': [39, 40],
 'Economics-1916-0-10.txt': [39],
 'Economics-1917-0-01.txt': [33]}
new_foot = {}
for file,vals in edited_foot_lines.items():
    if vals!=[]:
        
        new_foot[file] = min(vals)
        first_foot_line = min(vals)
        path = os.path.join(dest_dir,file)
        text = open(path,'r').read()
        lines = text.split("\n")
        lines_cut = lines[:first_foot_line]
        with open(path,'w') as f:
            f.write('\n'.join(lines_cut))
new_foot

{'Economics-1907-0-14.txt': 26,
 'Economics-1907-0-17.txt': 34,
 'Economics-1908-0-01.txt': 28,
 'Economics-1908-1-01.txt': 23,
 'Economics-1910-0-01.txt': 28,
 'Economics-1910-0-13.txt': 34,
 'Economics-1911-0-24.txt': 31,
 'Economics-1912-0-01.txt': 33,
 'Economics-1912-0-03.txt': 37,
 'Economics-1912-0-04.txt': 39,
 'Economics-1912-0-06.txt': 38,
 'Economics-1912-0-07.txt': 38,
 'Economics-1912-0-08.txt': 36,
 'Economics-1912-0-09.txt': 39,
 'Economics-1912-0-12.txt': 38,
 'Economics-1913-0-09.txt': 34,
 'Economics-1913-0-12.txt': 38,
 'Economics-1915-0-05.txt': 38,
 'Economics-1915-0-14.txt': 40,
 'Economics-1915-0-16.txt': 37,
 'Economics-1915-0-21.txt': 14,
 'Economics-1916-0-01.txt': 31,
 'Economics-1916-0-02.txt': 40,
 'Economics-1916-0-04.txt': 39,
 'Economics-1916-0-10.txt': 39,
 'Economics-1917-0-01.txt': 33}